<a href="https://colab.research.google.com/github/albert-melo/ProgCiencia-de-Dados/blob/master/WorkProgCienciaDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Trabalho da Disciplina Programação para Ciência de Dados

##Aluno:

> ###Albert Lima de Melo



#Montagem do Drive para acesso a base de dados
Criação do drive para acessar os dados a serem analisados e tratados.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Importando a base de dados **_dados-curso-completo.csv_** 
Utilizando a biblioteca do Pandas para leitura do arquivo de dados que estão tabulados no padrão CSV.

In [4]:
import pandas as pd
dados = pd.read_csv("/content/drive/My Drive/WorkProgCD/dados-curso-completo.csv")


#Visão Geral dos Dados

A seguir vamos explorar os dados da base disponibilizada

In [ ]:
dados.head()


##Dados Originais
###Requisito
1. Devem apresentar um resumo (sumário) com as estatísticas dos dados originais, ou seja, sem qualquer pré-processamento. A apresentação deste tipo de informação é relevante para que outras pessoas possam ter uma visão geral dos dados. As estatísticas podem ser apresentadas através de tabelas e/ou gráficos.


###Apresentando os dados iniciais

In [8]:
print("Colunas que compõem a base de dados: ")
info = dados.info()


Colunas que compõem a base de dados: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1658825 entries, 0 to 1658824
Data columns (total 13 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   data        1658825 non-null  object 
 1   usuario     1658768 non-null  object 
 2   apelido     1658825 non-null  object 
 3   texto       1658825 non-null  object 
 4   retweet     1658825 non-null  int64  
 5   seguidores  1658825 non-null  int64  
 6   idioma      1658825 non-null  object 
 7   lugar       74605 non-null    object 
 8   pais        74600 non-null    object 
 9   sigla       74595 non-null    object 
 10  latitude    8720 non-null     float64
 11  longitude   8720 non-null     float64
 12  hashtags    1658825 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 164.5+ MB


AttributeError: ignored

In [6]:
print("Resumo dos dados: ")
print( dados.describe().round(2))

Resumo dos dados: 
          retweet   seguidores  latitude  longitude
count  1658825.00   1658825.00   8720.00    8720.00
mean       197.85     32572.06    -40.24     -11.12
std       6337.64    368029.99     20.81      19.17
min          0.00         0.00   -123.12     -37.82
25%          0.00       101.00    -47.86     -23.31
50%          0.00       380.00    -43.96     -19.62
75%          0.00      1470.00    -38.45      -7.12
max    1139763.00  84241250.00    174.79      58.26


##Agrupando dados por países

In [ ]:
dados2 = dados
paises = dados2["pais"].drop_duplicates()
paises = list(paises)
selecao = dados2['pais'].isin(paises)
dados2 = dados2[selecao]

dados2.drop_duplicates()
grp_pais = dados2.groupby('pais')

In [ ]:
import matplotlib.pyplot as plt
plt.rc('figure', figsize = (20, 10))

In [ ]:
grp_pais[ ['pais'] ].count()

fig = grp_pais['pais'].count().plot.bar(color="blue")
fig.set_ylabel("Total de Postagens")
fig.set_xlabel("Países")
fig.set_title("Postagens por país", {'fontsize': 22})


###Quantidade de países e comentários com geolocalização

A seguir segue a quantidade de países identificados a partir das postagens realizadas e a quantidade da postagens postagens que possuem geolocalização.

In [ ]:
paises = dados["pais"].value_counts()
print("Quandidade de Países:", len(paises))
locais_geo = dados[ dados["latitude"].isna()==False]
print("Comentários com geo localização:", len(locais_geo))


##Instalação da bliblioteca _GeoPandas_
Utilizado para plotagem de dados utilizando geolocalização

In [ ]:
pip install geopandas

In [ ]:
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame 

In [ ]:
diferencaRegistrosLocais = len(dados) - len(locais)

print("Numero de registros sem geo localização: ", diferencaRegistrosLocais)

In [ ]:
geometry  = [Point(xy) for xy in zip(locais['latitude'], locais['longitude'])]
gdf = GeoDataFrame(locais, geometry = geometry,)

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(ax=world.plot(figsize=(20, 12)), marker = 'o', color = 'red', markersize = 15)

##Postagens

In [ ]:
registro = {'data':'', 'usuario':'', 'apelido':'', 'duvida':'', 'retweet':'', 'seguidores':'', 'idioma':'', 'lugar':'', 'pais':'', 'sigla': '', 'latitude':'' , 'longitude':'', 'hashtags':'' }

#Separa os textos que apresentam '?'
pre_perguntas = dados[dados["texto"].str.contains("?", regex=False) ]
#Converte conteúdo de texto em minúsculo
pre_perguntas["texto"].str.lower()

pre_perguntas.loc["texto" is in ["covid"]]


###Extraindo as perguntas do dataframe "pre_perguntas"

In [ ]:
type(pre_perguntas)

#Análise dos dados do DuvidasDB

In [ ]:
pre_perguntas.head()